In [1]:
#  Code ref: https://ritscm.regeneron.com/projects/DSEAIM/repos/secureaccess/browse/securekeyapp.py

In [1]:
!pip install --trusted-host pypi.python.org --trusted-host pypi.org --trusted-host files.pythonhosted.org --upgrade pip

  Obtaining dependency information for pip from https://files.pythonhosted.org/packages/15/aa/3f4c7bcee2057a76562a5b33ecbd199be08cdb4443a02e26bd2c3cf6fc39/pip-23.3.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 23.2.1
    Uninstalling pip-23.2.1:
      Successfully uninstalled pip-23.2.1


In [2]:
!pip install  --trusted-host pypi.python.org --trusted-host pypi.org --trusted-host files.pythonhosted.org \
msal \
retry \
simplejson \
openai \
grpcio==1.58.0 \
pymilvus==2.3.5 \
protobuf \
grpcio-tools==1.58.0 \
pymongo \
tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.8/81.8 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.9/174.9 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 23.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.0/99.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 680.8/680.8 kB 16.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 16.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━

In [3]:
import os
import openai
import requests
import simplejson as json
from retry import retry
from msal import PublicClientApplication, ConfidentialClientApplication, ClientApplication
import time

In [4]:
@retry (tries=3, delay=2)
def getapikey():
    # These are the AZURE parameters needed by the client application
    # In this scheme, the user does not have to worry about the api key, all of that is handled
    # at the AZURE Back End.   The API key is rotated hourly.
    # We can handle these as Kuberbetes secrets, or as environment variables.
    client_id = os.getenv("AZURE_CLIENT_ID")
    tenant_id = os.getenv("AZURE_TENANT_ID")
    endpoint = os.getenv("AZURE_ENDPOINT")

    scopes = [os.getenv("AZURE_APPLICATION_SCOPE")]

    app = ClientApplication(
        client_id=client_id,
        authority="https://login.microsoftonline.com/" + tenant_id
    )


    acquire_tokens_result = app.acquire_token_by_username_password(username=os.getenv("SVC_ACCOUNT"),
                                                                   password=os.getenv("SVC_PASSWORD"),
                                                                   scopes=scopes)
    if 'error' in acquire_tokens_result:
        print("Error: " + acquire_tokens_result['error'])
        print("Description: " + acquire_tokens_result['error_description'])
        return 2
    else:
        header_token = {"Authorization": "Bearer {}".format(acquire_tokens_result['access_token'])}
        rt = requests.post(url=endpoint, headers=header_token, data=b'{"key":"openaikey2"}')
        return rt.json()

In [5]:
openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_VERSION")
# We are dynamically getting the key from AZURE>   Access is based on the service account/ad group combination
openai.api_key = getapikey()
# print(openai.api_key)

In [6]:
import os
from openai import AzureOpenAI

ai_client = AzureOpenAI(
  api_key = getapikey(),  
  api_version = os.getenv("OPENAI_API_VERSION"),
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
)


## Create a function to load multiple proteins from the bioregistry mongo database

In [7]:
from pymongo import MongoClient
username = os.getenv("BIOREGISTRY_USER")
password = os.getenv("BIOREGISTRY_PASSWORD")
host1 = os.getenv("MONGOID_DATABASE_HOST1")
host2 = os.getenv("MONGOID_DATABASE_HOST2")
host3 = os.getenv("MONGOID_DATABASE_HOST3")
db = os.getenv("MONGOID_DATABASE")
mongo_client = MongoClient(f'mongodb://{username}:{password}@{host1}/{db}')
mongo_collection = mongo_client.get_database(db).get_collection('proteins')

In [8]:
from bson.json_util import dumps
# { "$and" : [ { "descriptive_name" : { "$ne" : "null" } }, { "descriptive_name" : { "$exists" : "true" } } ] }
proteins = mongo_collection.find({ "$and" : [ { "descriptive_name" : { "$ne" : None } }, { "descriptive_name" : { "$exists" : "true" } } ] }).limit(100)
for protein in proteins:
	print(dumps(protein))

{"_id": {"$oid": "581764631823767f32002797"}, "_type": "PesProtein", "regn_name": "REGN1487", "descriptive_name": "10xHis(M1-H11).OprF(K12-K360)", "eln_ref": "3184", "created_by": "michael.podgorski", "created_at": {"$date": "2011-04-08T00:00:00Z"}, "targets": [], "linked_pes_prgns": [], "updated_at": {"$date": "2021-04-14T08:20:43.907Z"}, "regn_number": 1487, "synonyms": [{"_id": {"$oid": "581764631823767f32002798"}, "type": "common_name", "value": "10xHis OprF"}], "protein_type": {"_id": {"$oid": "581764631823767f32002799"}, "name": "Custom Structure", "chains": 0, "unique_chains": 0, "svg": "\n                <svg xmlns=\"http://www.w3.org/2000/svg\" width=\"300\" height=\"300\">\n                <rect width=\"300\" height=\"300\" fill=\"#eee\"/>\n                <path transform=\"translate(-150,-50)\" d=\"m305.85 246.3h-31.922c-0.2229-0.74117-1.25352-3.44135-3.09194-8.1006-1.83846-4.65908-2.75768-8.73586-2.75764-12.2303-0.00004-6.67098 0.97488-13.0773 2.92479-19.219 1.94979-6.14154

## Connect to Milvus Vector Database
### Set up schema

In [9]:
from pymilvus import connections, utility, db
conn = connections.connect(host="standalone", port=19530, db_name="default")
db.list_database()

['default', 'bioregistry']

In [10]:
found = False
for database_name in db.list_database():
    if database_name == "bioregistry":
        found = True
if not found:
    db.create_database("bioregistry")

In [11]:
db.list_database()

['default', 'bioregistry']

In [12]:
db.using_database("bioregistry")

In [13]:
utility.list_collections()

['proteins']

In [14]:
new_collection_name = "proteins"

In [61]:
for collection_name in utility.list_collections():
    if collection_name == new_collection_name:
        utility.drop_collection(new_collection_name)

In [63]:
from pymilvus import CollectionSchema, FieldSchema, DataType

protein_id = FieldSchema(
  name="protein_id",
  dtype=DataType.INT64,
  is_primary=True,
)

name = FieldSchema(
  name="name",
  dtype=DataType.VARCHAR,
  max_length=32,
  # The default value will be used if this field is left empty during data inserts or upserts.
  # The data type of `default_value` must be the same as that specified in `dtype`.
  default_value="Unknown"
)

char_count = FieldSchema(
  name="char_count",
  dtype=DataType.INT64,
  # The default value will be used if this field is left empty during data inserts or upserts.
  # The data type of `default_value` must be the same as that specified in `dtype`.
  default_value=9999
)

protein_json = FieldSchema(
  name="protein_json",
  dtype=DataType.JSON,
  max_length=16000,
  # The default value will be used if this field is left empty during data inserts or upserts.
  # The data type of `default_value` must be the same as that specified in `dtype`.
  default_value="Unknown"
)

protein_vector = FieldSchema(
  name="protein_vector",
  dtype=DataType.FLOAT_VECTOR,
  dim=1536
)

schema = CollectionSchema(
  fields=[protein_id, name, char_count, protein_json, protein_vector],
  description="Protein search",
  enable_dynamic_field=True
)

In [64]:
from pymilvus import Collection
collection = Collection(
    name=new_collection_name,
    schema=schema,
    using='default',
    shards_num=2
    )

In [65]:
collection.create_partition("Disc1")
collection.has_partition("Disc1")

True

In [66]:
collection.create_partition("Disc2")
collection.has_partition("Disc2")

True

In [67]:
from pymilvus import Partition
disc2 = Partition(collection, name="Disc2")
disc2.release()
collection.drop_partition("Disc2")
collection.has_partition("Disc2")

False

In [68]:
utility.list_collections()

['proteins']

In [69]:
import tiktoken
def truncate_tokens(string: str, encoding_name: str, max_length: int = 8191) -> str:
    """Truncates a text string based on max number of tokens."""
    encoding = tiktoken.encoding_for_model(encoding_name)
    encoded_string = encoding.encode(string)
    num_tokens = len(encoded_string)

    if num_tokens > max_length:
        string = encoding.decode(encoded_string[:max_length])

    return string

## Insert proteins as vetcors
### { protein_id, name, char_count, protein_json, protein_vector 

In [70]:
from bson.json_util import dumps
from bson import json_util
import json
proteins = mongo_collection.find({"created_by": "kristin.hudson"})
i = 1
for protein in proteins:
	if 'descriptive_name'in protein:
		protein_json = json.dumps(protein,default=json_util.default)
		protein_descriptive_name= f"{protein['descriptive_name']}"
		#tokenized = truncate_tokens(string=protein_string, encoding_name="gpt-3.5-turbo", max_length=8191)
		print(protein_descriptive_name)
		response = ai_client.embeddings.create(
			input = protein_descriptive_name, #tokenized,
			model = "RegnADA002"
		)
		protein_id_column = [i]
		name_column = [protein['regn_name']]
		char_count_column = [len(protein_descriptive_name)]
		protein_json_column = [json.loads(protein_json)]
		protein_vector_column = [response.data[0].embedding]
		data = [protein_id_column, name_column, char_count_column, protein_json_column, protein_vector_column]
		collection.upsert(data)
		i += 1

mROR1(M1-A29).hBZLF1(S40-L48).GCGGS+G4Sx2.hB2m(I21-M119).G4Sx4.hHLA-E(G22-P301;Y105C).PADRE(A1-A13).mycmychis6
mROR1(M1-A29).6His.GCN4(R249-R281).G4Sx3.hGITR_ligand(E74-S199)
mROR1(M1-A29).B-gal(D96-V103).GCGGS+G4sx2.mB2m(I21-M119).G4sx4.mH2D(b)ecto(G25-V309;Y108C).PADRE(A1-A13).mycmychis6
mROR1(M1-A29).GFP(D118-L126).GCGGS+G4Sx2.mB2m(I21-M119).G4Sx4.mH2D(b)_ecto(G25-V309;Y108C).PADRE(A1-A13).mycmychis6
mROR1(M1-A29).hAPOC3(S21-A99).mycmychis6
mROR1(M1-A29).mAPOC3(E21-S99).mycmychis6
mROR1(M1-A29).MfAPOC3mature(S21-A99).mymychis
mROR1(M1-A29).hBTN3A1(Q30-G254).mycmychis6
mROR(M1-A29).hBTNL8_ecto(Q18-K238).mycmychis6
mROR1(M1-A29).mTIM-3ecto(R20-A193).hIgG1_Fc
mROR1(M1-A29).mTIM-3ecto(BALB_C)(R20-A193).hIgG1_Fc
mROR1(M1-A29).hUSP(I50-Y58).GCGGS+G4Sx2.hB2m(I21-M119).G4Sx4.hHLA-B*15_01-01-01(G25-P300;Y108C).mycmychis6
hgH_signal_sequence(M1-S23).hgH(R24-R717).mycmychis6.Furin-recognition-sequence(R1-R4).GSG.T2A(E1-P18).hgL_signal_sequence(M1-S30).hgL(A31-R278)
mROR1(M1-A29).VαhCD3E_HAAW5B

In [76]:

collection.compact()

In [77]:
index_params = {
  "metric_type":"COSINE",
  "index_type":"IVF_SQ8",
  "params":{"nlist":1024},
  
}
collection.create_index(
  field_name="protein_vector", 
  index_name="protein_vector_idx",
  index_params=index_params
)

Status(code=0, message=)

In [78]:
# index on a scalar field
collection.create_index(
  field_name="name", 
  index_name="protein_name_idx"
)

Status(code=0, message=)

## Search , Query and Hybrid

In [79]:
search_string = 'mROR1(M1-A29).hCD8'
response = ai_client.embeddings.create(
    input = search_string,
    model= "RegnADA002"
)
query_embedding = response.data[0].embedding
# query_embedding

In [80]:
search_params = {
    "metric_type": "COSINE", 
    "params": {"search_k": 64}
}

In [81]:
# Search
collection = Collection(new_collection_name)
utility.list_collections()
collection.load()
results =  collection.search(
    data=[query_embedding],
    anns_field="protein_vector",
    param=search_params,
    limit=10,
    expr=None,
    output_fields=['protein_json']
)
for result in results[0]:
    print(result)

id: 30, distance: 0.9395024180412292, entity: {'protein_json': {'_id': {'$oid': '581764721823767f3200445e'}, '_type': 'PesProtein', 'regn_name': 'REGN3912', 'descriptive_name': 'mROR1(M1-A29).hCD8b(L22-P170).mIgG2a_Fc', 'eln_ref': '201508250254', 'created_by': 'kristin.hudson', 'created_at': {'$date': '2015-08-27T00:00:00Z'}, 'targets': [], 'linked_pes_prgns': ['pRGN9057'], 'updated_at': {'$date': '2021-04-14T08:20:33.683Z'}, 'regn_number': 3912, 'synonyms': [{'_id': {'$oid': '581764721823767f3200445f'}, 'type': 'common_name', 'value': 'hCD8b ecto-mFc'}], 'protein_type': {'_id': {'$oid': '581764721823767f32004460'}, 'name': 'Custom Structure', 'chains': 0, 'unique_chains': 0, 'svg': '\n                <svg xmlns="http://www.w3.org/2000/svg" width="300" height="300">\n                <rect width="300" height="300" fill="#eee"/>\n                <path transform="translate(-150,-50)" d="m305.85 246.3h-31.922c-0.2229-0.74117-1.25352-3.44135-3.09194-8.1006-1.83846-4.65908-2.75768-8.73586-2.

In [82]:
query_expr = "protein_json['regn_name'] like 'REGN3853'" # Querying the json field

In [40]:
# Query
collection = Collection(new_collection_name)
utility.list_collections()
collection.load()
results = collection.query(
    expr = query_expr,
    offset = 0,
    limit = 10, 
    output_fields = ["name", "protein_json"]
)
results[0]

{'name': 'REGN3815',
 'protein_json': {'_id': {'$oid': '581764711823767f32004345'},
  '_type': 'PesProtein',
  'regn_name': 'REGN3815',
  'descriptive_name': 'mROR1(M1-A29).hAPOC3(S21-A99).mycmychis6',
  'eln_ref': '201506230035',
  'created_by': 'kristin.hudson',
  'created_at': {'$date': '2015-06-24T00:00:00Z'},
  'targets': [],
  'linked_pes_prgns': ['pRGN8899'],
  'updated_at': {'$date': '2021-04-14T08:20:33.405Z'},
  'regn_number': 3815,
  'synonyms': [{'_id': {'$oid': '581764711823767f32004346'},
    'type': 'common_name',
    'value': 'hAPOC3-mmh'}],
  'protein_type': {'_id': {'$oid': '581764711823767f32004347'},
   'name': 'Custom Structure',
   'chains': 0,
   'unique_chains': 0,
   'svg': '\n                <svg xmlns="http://www.w3.org/2000/svg" width="300" height="300">\n                <rect width="300" height="300" fill="#eee"/>\n                <path transform="translate(-150,-50)" d="m305.85 246.3h-31.922c-0.2229-0.74117-1.25352-3.44135-3.09194-8.1006-1.83846-4.65908-2.

In [83]:
# Hybrid : sama as search with expr provided
results =  collection.search(
    data=[query_embedding],
    anns_field="protein_vector",
    param=search_params,
    limit=5,
    expr=query_expr,
    output_fields=['protein_json']
)
for result in results[0]:
    print(result)

id: 14, distance: 0.9035544395446777, entity: {'protein_json': {'_id': {'$oid': '581764711823767f320043b4'}, '_type': 'PesProtein', 'regn_name': 'REGN3853', 'descriptive_name': 'mROR1(M1-A29).VαhCD3E_HAAW5B8-2_(2712N)_VH(E1-S123).G4Sx3.VαhCD3E_HAAW5B8-2_(2712N)_VK(E1-K107).G4Sx3.hIgG1Fc*(D104-K390;H318R;Y319F)', 'eln_ref': '1', 'created_by': 'kristin.hudson', 'created_at': {'$date': '2015-07-18T00:00:00Z'}, 'targets': [{'_id': {'$oid': '62839836820710025eed741c'}, 'target_id': 'C-TGT-99', 'target_name': 'CD3 (CD3E)', 'synonyms': None, 'source_type': 'TargetsDb', 'source_id': 'C-TGT-99'}], 'linked_pes_prgns': ['pRGN8966'], 'updated_at': {'$date': '2021-04-14T08:20:33.113Z'}, 'regn_number': 3853, 'synonyms': [{'_id': {'$oid': '581764711823767f320043b5'}, 'type': 'common_name', 'value': ' α-CD3E H1H2712N ScFv-hIgG1Fc'}], 'protein_type': {'_id': {'$oid': '5d07a077700f3e489862aae5'}, 'name': 'Standard mAb', 'chains': 4, 'unique_chains': 2, 'svg': '\n                <svg xmlns="http://www.w3